In [7]:
import os

import cv2
import imutils

import numpy as np

In [8]:
%load_ext watermark
%watermark -v -m -p cv2,imutils,numpy

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.5
IPython version      : 8.7.0

cv2    : 4.6.0
imutils: 0.5.4
numpy  : 1.23.5

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 69 Stepping 1, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [18]:
def read_image(file_name, margin = 1000):
    # Open image
    img = cv2.imread(file_name)
    if margin:
        img = cv2.copyMakeBorder(img, margin, margin, margin, margin, borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255))

    return img

def resize_image(img, height=1000):
    # Resize the image
    rsz = imutils.resize(img, height=height)

    # Calculate the ratio
    ratio = img.shape[0] / rsz.shape[0]
    
    return rsz, ratio

def rotate(img, angle, center = None, scale = 1.0):
    (h, w) = img.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rtt = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))

    return rtt

In [19]:
def onMouse(event, x, y, flags, param):
    img, pos_list = param
    if event == cv2.EVENT_LBUTTONDOWN:
        pos_list.append((x, y))
        cv2.circle(img, pos_list[-1], 5, (0,0,255), -1)

In [20]:
def cv_crop_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 0)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)
    
    image = rsz.copy()
    pos_list = []
    
    rect_rsz = None
    label = label+' ~ Cropping'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse, param=(image, pos_list))
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            pos_list = []
            image = rsz.copy()
            cv2.setMouseCallback(label, onMouse, param=(image, pos_list))
        elif key in (13, 32): # Enter or Space
            if verbose: print(pos_list)
            rect_rsz = np.asarray(pos_list).reshape([-1, 1, 2])
            break
    cv2.destroyAllWindows()
    
    rect_img = ((rect_rsz.astype('float'))*ratio).astype('int')
    x, y, w, h = cv2.boundingRect(rect_img)
    img = img[y:y+h, x:x+w, :]
    
    return img

def cv_rotate_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21 , 2)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)

    image = rsz.copy()
    pos_list = []
    
    left, right = None, None
    label = label+' ~ Angle'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse, param=(image, pos_list))
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            pos_list.clear()
            image[:,:,:] = rsz
        elif key in (13, 32): # Enter or Space
            if len(pos_list) < 2:
                continue
            if verbose: print(pos_list)
            left, right = pos_list[:2]
            break
    cv2.destroyAllWindows()
    
    angle = np.arctan( (right[1]-left[1])/(right[0]-left[0]) ) * 180 / np.pi
    if verbose: print(angle)
        
    img = rotate(img, angle)
    
    return img

def cv_save_image(img, file_name, label, verbose=False):
    rsz, ratio = resize_image(img, height=800)
    done = False
    
    label = label+' ~ Result'
    cv2.namedWindow(label)

    cv2.imshow(label, rsz)

    key = cv2.waitKey() & 0xFF
    if key == 27: # Escape
        done = False
    elif key in (13, 32): # Enter or Space
        cv2.imwrite(file_name, img)
        done = True
        
    cv2.destroyAllWindows()
    
    return done

In [21]:
def cv2_manual_photo(file_name, iputdir='', oputdir='', label=None):
    oputdir = oputdir or iputdir
    img = read_image(os.path.join(iputdir, file_name), margin=0)

    while True:
        img_proc = cv_crop_image(img, label=label, threshold=True)

        img_proc = cv_rotate_image(img_proc, label=label, threshold=True)

        img_proc = cv_crop_image(img_proc, label=label, threshold=True)
        
        if cv_save_image(img_proc, os.path.join(oputdir, file_name.replace('_orig.png', '.png')), label=label):
            break

In [22]:
iputdir = 'photo_2_crop'

png_name_list = [png for png in os.listdir(iputdir) if png.endswith('_orig.png')]
for i, png_name in enumerate(png_name_list, start=1):
    cv2_manual_photo(png_name, iputdir, label='[{} of {}] {}'.format(i, len(png_name_list), png_name))
        
print("Done!")

Done!
